# Q* Learning with FrozenLake 🕹️⛄
<br> 
In this Notebook, we'll implement an agent <b>that plays FrozenLake.</b>

The goal of this game is <b>to go from the starting state (S) to the goal state (G)</b> by walking only on frozen tiles (F) and avoid holes (H).However, the ice is slippery, <b>so you won't always move in the direction you intend (stochastic environment)</b>

## Step 0: Import the dependencies 📚
We use 3 libraries:
- `Numpy` for our Qtable
- `OpenAI Gym` for our FrozenLake Environment
- `Random` to generate random numbers

In [1]:
import numpy as np
import gym
import random

## Step 1: Create the environment 🎮
- Here we'll create the FrozenLake environment. 
- OpenAI Gym is a library <b> composed of many environments that we can use to train our agents.</b>
- In our case we choose to use Frozen Lake.

In [2]:
env = gym.make("FrozenLake-v1",map_name="8x8")

## Step 2: Create the Q-table and initialize it 🗄️
- Now, we'll create our Q-table, to know how much rows (states) and columns (actions) we need, we need to calculate the action_size and the state_size
- OpenAI Gym provides us a way to do that: `env.action_space.n` and `env.observation_space.n`

In [3]:
action_size = env.action_space.n
state_size = env.observation_space.n
print(action_size)
print(state_size)

4
64


In [4]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Step 3: Create the hyperparameters ⚙️
- Here, we'll specify the hyperparameters

In [5]:
total_episodes = 30000        # Total episodes
learning_rate = 0.1           # Learning rate
max_steps = 300                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.0001             # Exponential decay rate for exploration prob

## Step 4: The Q learning algorithm 🧠
- Now we implement the Q learning algorithm:

In [6]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()[0]
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, truncated,info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state,:]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

Score over time: 0.20966666666666667
[[4.39800822e-02 4.40579537e-02 4.49290822e-02 4.41871393e-02]
 [4.91338305e-02 4.84462610e-02 5.42690811e-02 4.90907623e-02]
 [5.74091789e-02 5.60049044e-02 6.41377231e-02 5.97059028e-02]
 [7.65018922e-02 7.38113338e-02 8.26615142e-02 7.76161687e-02]
 [8.77434700e-02 9.12792518e-02 1.09622155e-01 9.12222310e-02]
 [1.05620325e-01 1.13018973e-01 1.34197530e-01 1.09297615e-01]
 [1.25850261e-01 1.28474349e-01 1.42174467e-01 1.26304145e-01]
 [1.44155858e-01 1.31470137e-01 1.38432699e-01 1.31018474e-01]
 [4.05684442e-02 3.96327401e-02 4.13746839e-02 4.39544031e-02]
 [4.56763886e-02 4.63579608e-02 5.18081722e-02 4.65164137e-02]
 [4.98506226e-02 5.22753229e-02 5.89148218e-02 6.22922882e-02]
 [4.41328467e-02 4.65925413e-02 4.71730799e-02 8.07509325e-02]
 [8.16583759e-02 8.34152619e-02 1.13853926e-01 9.04199376e-02]
 [1.05948537e-01 1.06610103e-01 1.37263238e-01 1.06211994e-01]
 [1.40712806e-01 1.45126595e-01 1.53117279e-01 1.36214370e-01]
 [1.48073205e-01 1

## Step 5: Use our Q-table to play FrozenLake ! 👾

In [7]:
env.reset()

result = 0
for episode in range(1000):
    state = env.reset()[0]
    step = 0
    done = False

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            result += reward
            break
        state = new_state

print("Completionrate",result/1000)
env.close()

Completionrate 0.574


<h1>Run Game with rendering</h1>

In [ ]:
env = gym.make("FrozenLake-v1",render_mode="human",map_name="8x8")
env.reset()

result = 0
for episode in range(5):
    state = env.reset()[0]
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode+1)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            result += reward
            # We print the number of step it took.
            print("Number of steps", step)
            print("Reward",reward)
            break
        state = new_state

print("Completionrate",result/5)
env.close()

****************************************************
EPISODE  1
